**keyBERT** - Termextraktion mit BERT

Quellen: https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea und https://github.com/MaartenGr/KeyBERT

Wiederholung: BERT stellt Wörter als Vektoren dar. BERT sieht Wörter also nicht als feste Strings, sondern erkennt, dass Begriffe wie "See", "Teich" und "Gewässer" eine Ähnlichkeit aufweisen.

Wir können den folgenden Beispieltext verwenden.

In [1]:
doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """

**Preprocessing**

scikit-learn ist eine Sammlung von Funktionen für maschinelles Lernen mit Python. [1]

Wir können die Methode "CountVectorizer" aus scikit-learn verwenden. Sie überführt die Tokens eines Textdokuents in eine Matrix mit Wortzählungen. [2]

Der Wert "n_gram_range" kann angepasst werden, um die Länge der später extrahierten Keywords festzulegen.

Die Variable "candidates" enthält eine Liste von extrahierten "Termen" - sie heißt candidates, weil sie nur automatisch extrahierte Wörter enthält, für die manuell überprüft werden muss, ob sie tatsächlich Terme darstellen.

stop_words entfernt Stoppwörter (I, be, the, in, a, und so weiter)

[1] https://scikit-learn.org/stable

[2] https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 1)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names()

Anschließend konvertieren wir die Wörter des Textdokuments mit BERT in Vektoren.

sentence-transformers ist eine Python-Bibliothek, mit der Wörter über ein vortrainiertes BERT-Modell in eine Vektor-Darstellung überführt werden können.

Im oberen Code-Feld wird die Bibliothek installiert.

Der untere Code zeigt, dass wir tatsächlich nicht BERT verwenden, sondern distilBERT [1]. Dabei handelt es sich um ein Modell, das ähnlich effektiv wie BERT ist, aber dabei einen wesentlich kleineres Aufbau hat, wodurch sich die Anwendung beschleunigt.

Das gesamte Dokument wird als ein (!) Vektor dargestellt ("doc embedding").
Die "Kandidaten" werden jeweils als Vektor dargestellt ("candidate embeddings").

[1] Sanh, V., Debut, L., Chaumond, J. & Wolf, T. (2019). Distilbert, a distilled version of BERT: smaller, faster, cheaper and lighter. In Proceedings of the NeurIPS EMC2 Workshop.

In [6]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=65bb64a56e389376e28da92870e5b5f9e8e2140d9770628fd781399ac34854a1
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Wenn wir die "candidate embeddings" ausgeben, erhalten wir Zahlenwerte.

In [8]:
for entry in candidate_embeddings:
    print (entry)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
 -1.5056562   0.10967835 -0.84166735 -0.61423904  0.03923832 -0.01318826
  0.2848045  -0.01452677  0.13147882 -1.3431815  -0.09165961  0.96500283
  1.0334729  -0.2503225   0.05338562  0.58870786 -1.4141334  -0.32669875
  0.9884693   0.6819753   0.20780487  0.3777089  -0.58863574  0.4255216
 -0.28534305  0.14975035  0.02314212 -0.40491357 -0.21634899 -0.17543994
 -0.43252337 -0.47132024  1.8530239   0.15163994 -0.02079859  0.6756819
  0.5574327  -0.3862858  -0.49201453 -0.5485964   0.19703875  0.11984351
  0.28075612 -0.53300184 -0.24491592 -0.5958145  -0.36197016 -0.4941968
  0.16456674  0.97635573  0.8962762  -0.7392092  -0.21147464  0.6601911
 -0.34644005  0.17112263  0.79303855 -0.06634168  0.21206348  0.5390217
 -0.2196731   0.39442912  0.32231155 -0.09508827  0.24321534  1.0669454
 -0.3056119  -1.9571714  -0.17117672 -0.32674637  1.0503199  -0.4661046
  0.91423863  0.199616   -0.4496735  -0.33074287 -0.3031175   0.

Jetzt können wir die einzelnen Wörter (die "Kandidaten") mit dem Gesamttext vergleichen.

Das ist schon alles. Mit top_n können wir ausgeben, wie viele Wörter im Ranking der Wörter, die dem Dokument am ähnlichsten sind, ausgegeben werden.

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

for keyword in keywords:
    print (keyword)

mapping
class
training
algorithm
learning


Die Funktionen werden in der Klasse "keybert" bereitgestellt. [1]

Wir können sie hier viel einfacher verwenden.

Sie wird im folgenden Code importiert (!pip install keybert). Die Variable "doc" ist eine Wiederholung von oben - sie wird hier nur zur Übersichtlichkeit wiederholt.

[1] https://github.com/MaartenGr/KeyBERT

In [17]:
!pip install keybert

from keybert import KeyBERT

doc = """
         Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal).
         A supervised learning algorithm analyzes the training data and produces an inferred function,
         which can be used for mapping new examples. An optimal scenario will allow for the
         algorithm to correctly determine the class labels for unseen instances. This requires
         the learning algorithm to generalize from the training data to unseen situations in a
         'reasonable' way (see inductive bias).
      """
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)

for keyword in keywords:
    print (keyword)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
('supervised', 0.6676)
('labeled', 0.4896)
('learning', 0.4813)
('training', 0.4134)
('labels', 0.3947)


Wir können im Textdokument (bisher: Die Variable "doc") die Wörter hervorheben, die als Keywords erkannt werden.

In [19]:
keywords = kw_model.extract_keywords(doc, highlight=True)

Supervised learning is the machine learning task of learning function that maps an input to an output based on 
example input output pairs It infers function from labeled training data consisting of set of training examples In 
supervised learning each example is pair consisting of an input object typically vector and desired output value 
also called the supervisory signal supervised learning algorithm analyzes the training data and produces an 
inferred function which can be used for mapping new examples An optimal scenario will allow for the algorithm to 
correctly determine the class labels for unseen instances This requires the learning algorithm to generalize from 
the training data to unseen situations in reasonable way see inductive bias